# EBAC - Regressão II - regressão múltipla

## Tarefa I

#### Previsão de renda

Vamos trabalhar com a base 'previsao_de_renda.csv', que é a base do seu próximo projeto. Vamos usar os recursos que vimos até aqui nesta base.

|variavel|descrição|
|-|-|
|data_ref                | Data de referência de coleta das variáveis |
|index                   | Código de identificação do cliente|
|sexo                    | Sexo do cliente|
|posse_de_veiculo        | Indica se o cliente possui veículo|
|posse_de_imovel         | Indica se o cliente possui imóvel|
|qtd_filhos              | Quantidade de filhos do cliente|
|tipo_renda              | Tipo de renda do cliente|
|educacao                | Grau de instrução do cliente|
|estado_civil            | Estado civil do cliente|
|tipo_residencia         | Tipo de residência do cliente (própria, alugada etc)|
|idade                   | Idade do cliente|
|tempo_emprego           | Tempo no emprego atual|
|qt_pessoas_residencia   | Quantidade de pessoas que moram na residência|
|renda                   | Renda em reais|

In [17]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import statsmodels.formula.api as smf
import statsmodels.api as sm
import patsy 
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)


In [18]:
df = pd.read_csv('previsao_de_renda.csv')

In [19]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15000 entries, 0 to 14999
Data columns (total 15 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Unnamed: 0             15000 non-null  int64  
 1   data_ref               15000 non-null  object 
 2   id_cliente             15000 non-null  int64  
 3   sexo                   15000 non-null  object 
 4   posse_de_veiculo       15000 non-null  bool   
 5   posse_de_imovel        15000 non-null  bool   
 6   qtd_filhos             15000 non-null  int64  
 7   tipo_renda             15000 non-null  object 
 8   educacao               15000 non-null  object 
 9   estado_civil           15000 non-null  object 
 10  tipo_residencia        15000 non-null  object 
 11  idade                  15000 non-null  int64  
 12  tempo_emprego          12427 non-null  float64
 13  qt_pessoas_residencia  15000 non-null  float64
 14  renda                  15000 non-null  float64
dtypes:

1. Ajuste um modelo para prever log(renda) considerando todas as covariáveis disponíveis.
    - Utilizando os recursos do Patsy, coloque as variáveis qualitativas como *dummies*.
    - Mantenha sempre a categoria mais frequente como casela de referência
    - Avalie os parâmetros e veja se parecem fazer sentido prático.  


2. Remova a variável menos significante e analise:
    - Observe os indicadores que vimos, e avalie se o modelo melhorou ou piorou na sua opinião.
    - Observe os parâmetros e veja se algum se alterou muito.  


3. Siga removendo as variáveis menos significantes, sempre que o *p-value* for menor que 5%. Compare o modelo final com o inicial. Observe os indicadores e conclua se o modelo parece melhor. 
    

In [20]:
# 1. 
df.drop(['Unnamed: 0', 'data_ref', 'id_cliente'], axis=1, inplace=True)
df.head()

,sexo,posse_de_veiculo,posse_de_imovel,qtd_filhos,tipo_renda,educacao,estado_civil,tipo_residencia,idade,tempo_emprego,qt_pessoas_residencia,renda
0,F,False,True,0,Empresário,Secundário,Solteiro,Casa,26,6.602740,1.0,8060.34
1,M,True,True,0,Assalariado,Superior completo,Casado,Casa,28,7.183562,2.0,1852.15
2,F,True,True,0,Empresário,Superior completo,Casado,Casa,35,0.838356,2.0,2253.89
3,F,False,True,1,Servidor público,Superior completo,Casado,Casa,30,4.846575,3.0,6600.77
4,M,True,False,0,Assalariado,Secundário,Solteiro,Governamental,33,4.293151,1.0,6475.97


In [21]:
for col in df.columns:
    if col == 'renda':
        continue
    print(f'{df[col].name}: {df[col].value_counts().index[0]}')

sexo: F
posse_de_veiculo: False
posse_de_imovel: True
qtd_filhos: 0
tipo_renda: Assalariado
educacao: Secundário
estado_civil: Casado
tipo_residencia: Casa
idade: 40
tempo_emprego: 4.216438356164384
qt_pessoas_residencia: 2.0


In [22]:
y, X = patsy.dmatrices('''
                       np.log(renda) ~ 
                               C(sexo, Treatment(0))
                               + C(tipo_renda, Treatment(0))
                               + C(educacao, Treatment(2))
                               + C(estado_civil, Treatment(0))
                               + C(tipo_residencia, Treatment(1))
                               + C(posse_de_veiculo, Treatment(0))
                               + C(posse_de_imovel, Treatment(1))
                               + qtd_filhos
                               + idade
                               + tempo_emprego
                               + qt_pessoas_residencia
                       ''', data = df)

In [23]:
sm.OLS(y, X).fit().summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          np.log(renda)   R-squared:                       0.357
Model:                            OLS   Adj. R-squared:                  0.356
Method:                 Least Squares   F-statistic:                     287.5
Date:                Fri, 16 Jun 2023   Prob (F-statistic):               0.00
Time:                        15:08:49   Log-Likelihood:                -13568.
No. Observations:               12427   AIC:                         2.719e+04
Df Residuals:                   12402   BIC:                         2.737e+04
Df Model:                          24                                         
Covariance Type:            nonrobust                                         
=====================================================================================================================
                                                        coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------------------------------
Intercept                                             6.6093      0.219     30.214      0.000       6.181       7.038
C(sexo, Treatment(0))[T.M]                            0.7874      0.015     53.723      0.000       0.759       0.816
C(tipo_renda, Treatment(0))[T.Bolsista]               0.2209      0.241      0.916      0.360      -0.252       0.694
C(tipo_renda, Treatment(0))[T.Empresário]             0.1551      0.015     10.387      0.000       0.126       0.184
C(tipo_renda, Treatment(0))[T.Pensionista]           -0.3087      0.241     -1.280      0.201      -0.782       0.164
C(tipo_renda, Treatment(0))[T.Servidor público]       0.0576      0.022      2.591      0.010       0.014       0.101
C(educacao, Treatment(2))[T.Primário]                 0.0141      0.072      0.196      0.844      -0.127       0.155
C(educacao, Treatment(2))[T.Pós graduação]            0.1212      0.142      0.853      0.394      -0.157       0.400
C(educacao, Treatment(2))[T.Superior completo]        0.1079      0.014      7.763      0.000       0.081       0.135
C(educacao, Treatment(2))[T.Superior incompleto]     -0.0290      0.032     -0.900      0.368      -0.092       0.034
C(estado_civil, Treatment(0))[T.Separado]             0.3265      0.111      2.935      0.003       0.108       0.545
C(estado_civil, Treatment(0))[T.Solteiro]             0.2672      0.109      2.453      0.014       0.054       0.481
C(estado_civil, Treatment(0))[T.União]               -0.0343      0.025     -1.361      0.174      -0.084       0.015
C(estado_civil, Treatment(0))[T.Viúvo]                0.3689      0.116      3.191      0.001       0.142       0.596
C(tipo_residencia, Treatment(1))[T.Aluguel]           0.0442      0.054      0.815      0.415      -0.062       0.150
C(tipo_residencia, Treatment(1))[T.Com os pais]       0.0184      0.029      0.625      0.532      -0.039       0.076
C(tipo_residencia, Treatment(1))[T.Comunitário]      -0.0754      0.091     -0.829      0.407      -0.254       0.103
C(tipo_residencia, Treatment(1))[T.Estúdio]           0.1107      0.084      1.319      0.187      -0.054       0.275
C(tipo_residencia, Treatment(1))[T.Governamental]    -0.0129      0.039     -0.331      0.741      -0.089       0.064
C(posse_de_veiculo, Treatment(0))[T.True]             0.0441      0.014      3.119      0.002       0.016       0.072
C(posse_de_imovel, Treatment(1))[T.False]            -0.0829      0.014     -5.926      0.000      -0.110      -0.055
qtd_filhos                                           -0.2658      0.108     -2.463      0.014      -0.477      -0.054
idade                                                 0.0053      0.001      6.624      0.000       0.004       0.007
tempo_emprego  

#### Observando o P-value,  a variável 'tipo_residencia', deve ser descartada por ser estatísticamente menos significante

In [29]:
# 2. 
y, X = patsy.dmatrices('''
                       np.log(renda) ~ 
                               C(sexo, Treatment(1))
                               + C(tipo_renda, Treatment(0))
                               + C(educacao, Treatment(2))
                               + C(estado_civil, Treatment(0))
                               + C(posse_de_veiculo, Treatment(0))
                               + C(posse_de_imovel, Treatment(1))
                               + qtd_filhos
                               + idade
                               + tempo_emprego
                               + qt_pessoas_residencia
                       ''', data = df)

In [30]:
sm.OLS(y, X).fit().summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          np.log(renda)   R-squared:                       0.357
Model:                            OLS   Adj. R-squared:                  0.356
Method:                 Least Squares   F-statistic:                     363.0
Date:                Fri, 16 Jun 2023   Prob (F-statistic):               0.00
Time:                        15:22:48   Log-Likelihood:                -13569.
No. Observations:               12427   AIC:                         2.718e+04
Df Residuals:                   12407   BIC:                         2.733e+04
Df Model:                          19                                         
Covariance Type:            nonrobust                                         
====================================================================================================================
                                                       coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------------------------------
Intercept                                            7.4026      0.219     33.872      0.000       6.974       7.831
C(sexo, Treatment(1))[T.F]                          -0.7893      0.015    -53.999      0.000      -0.818      -0.761
C(tipo_renda, Treatment(0))[T.Bolsista]              0.2219      0.241      0.920      0.357      -0.251       0.695
C(tipo_renda, Treatment(0))[T.Empresário]            0.1556      0.015     10.433      0.000       0.126       0.185
C(tipo_renda, Treatment(0))[T.Pensionista]          -0.3116      0.241     -1.292      0.196      -0.784       0.161
C(tipo_renda, Treatment(0))[T.Servidor público]      0.0589      0.022      2.653      0.008       0.015       0.102
C(educacao, Treatment(2))[T.Primário]                0.0097      0.072      0.135      0.893      -0.131       0.150
C(educacao, Treatment(2))[T.Pós graduação]           0.1200      0.142      0.845      0.398      -0.159       0.399
C(educacao, Treatment(2))[T.Superior completo]       0.1077      0.014      7.761      0.000       0.080       0.135
C(educacao, Treatment(2))[T.Superior incompleto]    -0.0288      0.032     -0.893      0.372      -0.092       0.034
C(estado_civil, Treatment(0))[T.Separado]            0.3272      0.111      2.942      0.003       0.109       0.545
C(estado_civil, Treatment(0))[T.Solteiro]            0.2683      0.109      2.464      0.014       0.055       0.482
C(estado_civil, Treatment(0))[T.União]              -0.0329      0.025     -1.310      0.190      -0.082       0.016
C(estado_civil, Treatment(0))[T.Viúvo]               0.3678      0.116      3.182      0.001       0.141       0.594
C(posse_de_veiculo, Treatment(0))[T.True]            0.0442      0.014      3.125      0.002       0.016       0.072
C(posse_de_imovel, Treatment(1))[T.False]           -0.0819      0.014     -5.966      0.000      -0.109      -0.055
qtd_filhos                                          -0.2655      0.108     -2.460      0.014      -0.477      -0.054
idade                                                0.0052      0.001      6.579      0.000       0.004       0.007
tempo_emprego                                        0.0617      0.001     59.485      0.000       0.060       0.064
qt_pessoas_residencia                                0.2923      0.108      2.713      0.007       0.081       0.504
==============================================================================
Omnibus:                        0.825   Durbin-Watson:                   2.023
Prob(Omnibus):                  0.662   Jarque-Bera (JB):                0.809
Skew:                           0.019   Prob(JB):                        0.667
Kurtosis:                       3.009   Cond. No.                     2.13e+

#### Removendo a variável menos significante, pouca coisa mudou, o R² continua o mesmo e alguns P-values aumentaram, mas só um pouco, nada significativo

In [35]:
# 3. 
y, X = patsy.dmatrices('''
                       np.log(renda) ~ 
                               C(sexo, Treatment(0))
                               + C(posse_de_veiculo, Treatment(0))
                               + C(posse_de_imovel, Treatment(1))
                               + qtd_filhos
                               + idade
                               + tempo_emprego
                               + qt_pessoas_residencia
                       ''', data = df)

In [36]:
sm.OLS(y, X).fit().summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          np.log(renda)   R-squared:                       0.347
Model:                            OLS   Adj. R-squared:                  0.346
Method:                 Least Squares   F-statistic:                     940.8
Date:                Fri, 16 Jun 2023   Prob (F-statistic):               0.00
Time:                        15:30:46   Log-Likelihood:                -13672.
No. Observations:               12427   AIC:                         2.736e+04
Df Residuals:                   12419   BIC:                         2.742e+04
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
=============================================================================================================
                                                coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------------------------
Intercept                                     7.3243      0.044    167.889      0.000       7.239       7.410
C(sexo, Treatment(0))[T.M]                    0.7694      0.015     52.676      0.000       0.741       0.798
C(posse_de_veiculo, Treatment(0))[T.True]     0.0569      0.014      4.022      0.000       0.029       0.085
C(posse_de_imovel, Treatment(1))[T.False]    -0.0866      0.014     -6.275      0.000      -0.114      -0.060
qtd_filhos                                    0.0338      0.019      1.735      0.083      -0.004       0.072
idade                                         0.0049      0.001      6.408      0.000       0.003       0.006
tempo_emprego                                 0.0610      0.001     59.075      0.000       0.059       0.063
qt_pessoas_residencia                        -0.0092      0.016     -0.566      0.572      -0.041       0.023
==============================================================================
Omnibus:                        1.240   Durbin-Watson:                   2.025
Prob(Omnibus):                  0.538   Jarque-Bera (JB):                1.213
Skew:                           0.022   Prob(JB):                        0.545
Kurtosis:                       3.019   Cond. No.                         301.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

#### Apesar de uma pequena redução nos valores de R² e R² ajustado, e um aumento no AIC, é importante notar que essas mudanças foram insignificantes considerando a quantidade de variáveis removidas do modelo. Isso indica que o modelo refinado é significativamente melhor do que o modelo inicial, pois a remoção das variáveis não estatisticamente relevantes evita problemas de overfitting. De acordo com o princípio da Navalha de Occam, podemos concluir que o modelo refinado, por ser mais simples, é preferível ao modelo inicial, uma vez que as métricas avaliadas apresentaram pouca alteração após as simplificações realizadas.